In [190]:
import pandas as pd
import numpy as np

In [191]:
df = pd.read_excel('BIEU THUE XNK 2016.xlsx', sheet_name="NHAP KHAU (FULL)", header=5, skiprows=range(6,19), converters={'hs':str}).fillna("")

# Fix heading (02.05	Thịt ngựa, lừa, la, tươi, ướp lạnh hoặc đông lạnh.)

In [192]:
def insert_row(df, pos, row):
    df2 = pd.DataFrame(np.insert(df.values, pos, values=row, axis=0))
    df2.columns = df.columns
    return df2

In [193]:
# 2502.00.00

def fix_heading(df):
    for index, row in df.iterrows():
        if len(str(row['hs'])) > 5 and not row['description'].startswith('-'):
            new_row = row.copy()
            new_row['hs'] = row['hs'][:2] + '.' + row['hs'][2:4]
            df['description'][index] = "- " + row['description']
            return fix_heading(insert_row(df, index, new_row))
    return df


fixed_heading = fix_heading(df)
fixed_heading.to_excel('fixed_heading.xlsx', index=False)

# Remove section, chapter, all notes, general

In [ ]:
def removeStuff(fixed_heading):
    remove_index = []
    for index, row in fixed_heading.iterrows():
        if not row['hs'] and not row['description'].startswith('- '):
            remove_index.append(index)
    return fixed_heading.drop(remove_index)


removed_df = removeStuff(fixed_heading).reset_index(drop=True)
removed_df["id"] = removed_df.index + 1
cols = removed_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
removed_df[cols].to_excel('removed_df.xlsx', index=False)